# Gold EDA

This notebook conducts EDA aimed at generating a gold layer of data, which is considered ready for model consumption. EDA in this notebook will analyze the various predictor variables in the context of the "binds" target variable. It will also analyze some options for converting the "molecule_smiles" string representation of a molecule into a numeric representation that can be better consumed by a model. 

Recall this project uses a psuedo [medallion](https://www.databricks.com/glossary/medallion-architecture) where the gold layer is model ready data. 